In [51]:
# Importando bibliotecas necessárias para a construção de grafos
import netpixi
from netpixi.integration.gt import *
import pandas as pd
import numpy as np
import csv
import ast
import cpnet
from graph_tool import spectral

In [2]:
df = pd.read_csv('./Video_Games_Sales_as_at_22_Dec_2016.csv')
df.dropna(subset=['Publisher'], inplace=True)
df = df.loc[~df['Critic_Score'].isna() & ~df['User_Score'].isna() & ~df['Rating'].isna()]

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6943 entries, 0 to 16706
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             6943 non-null   object 
 1   Platform         6943 non-null   object 
 2   Year_of_Release  6825 non-null   float64
 3   Genre            6943 non-null   object 
 4   Publisher        6943 non-null   object 
 5   NA_Sales         6943 non-null   float64
 6   EU_Sales         6943 non-null   float64
 7   JP_Sales         6943 non-null   float64
 8   Other_Sales      6943 non-null   float64
 9   Global_Sales     6943 non-null   float64
 10  Critic_Score     6943 non-null   float64
 11  Critic_Count     6943 non-null   float64
 12  User_Score       6943 non-null   float64
 13  User_Count       6943 non-null   float64
 14  Developer        6943 non-null   object 
 15  Rating           6943 non-null   object 
dtypes: float64(10), object(6)
memory usage: 922.1+ KB


In [4]:
df.groupby('Genre')['Name'].count().sort_values(ascending=False)

Genre
Action          1663
Sports           965
Shooter          882
Role-Playing     718
Racing           591
Platform         406
Misc             394
Fighting         381
Simulation       301
Strategy         272
Adventure        249
Puzzle           121
Name: Name, dtype: int64

In [5]:
df_grouped_Genre = df.groupby(['Genre']).agg({'Global_Sales': np.mean})
df_grouped_Genre.sort_values('Global_Sales', ascending=False)

,Global_Sales
Genre,
Misc,1.077741
Shooter,0.934025
Platform,0.931429
Sports,0.881513
Racing,0.811844
Action,0.736091
Role-Playing,0.701100
Simulation,0.676146
Fighting,0.658110


In [6]:
sports_mean_sales = df_grouped_Genre.loc['Sports']['Global_Sales']
action_mean_sales = df_grouped_Genre.loc['Action']['Global_Sales'] 
rpg_mean_sales = df_grouped_Genre.loc['Role-Playing']['Global_Sales'] 

In [7]:
def segregate_genre_to_list(df, genre, df_list, threshold=None):
    seg_df = df[df['Genre'] == genre]
    if threshold:
        seg_df = seg_df[seg_df['Global_Sales'] > threshold]
    df_list.append(seg_df)

In [27]:
# sorted_years = df['Year_of_Release'].unique()
# sorted_years.sort()
sorted_years = list(range(1985, 2017))
print(sorted_years)

[1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]


In [28]:
df_list = []
sport_df_list = []
action_df_list = []
rpg_df_list = []
margem = 4
for i in range(0, len(sorted_years), margem):
    df_list.append(df[df['Year_of_Release'].isin(sorted_years[i:i+margem])])


In [43]:
df_list[2]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
153,Tekken 2,PS,1996.0,Fighting,Sony Computer Entertainment,2.26,1.89,1.36,0.23,5.74,89.0,8.0,8.9,102.0,Namco,T
203,Resident Evil,PS,1996.0,Action,Virgin Interactive,2.05,1.16,1.11,0.73,5.05,91.0,8.0,9.0,266.0,Capcom,M
229,Tomb Raider,PS,1996.0,Action,Eidos Interactive,2.29,1.97,0.13,0.24,4.63,91.0,13.0,8.6,147.0,Core Design Ltd.,T
903,PaRappa The Rapper,PS,1996.0,Misc,Sony Computer Entertainment,0.26,0.16,1.46,0.03,1.92,92.0,15.0,7.4,48.0,NanaOn-Sha,K-A
1160,Diablo,PC,1996.0,Role-Playing,Activision,0.01,1.58,0.00,0.00,1.59,94.0,12.0,8.7,850.0,Blizzard Entertainment,M
1567,Battle Arena Toshinden,PS,1994.0,Fighting,Sony Computer Entertainment,0.39,0.26,0.53,0.08,1.27,69.0,4.0,6.3,4.0,Tamsoft,T
2014,Tomb Raider,PC,1996.0,Action,Eidos Interactive,0.96,0.07,0.00,0.00,1.03,86.0,18.0,8.5,3552.0,"Crystal Dynamics, Nixxes Software",M
9215,Super Puzzle Fighter II Turbo,PS,1996.0,Puzzle,Virgin Interactive,0.08,0.05,0.00,0.01,0.14,83.0,9.0,7.4,5.0,Capcom,E


In [30]:
publisher_platforms = []
for db in df_list:
    publisher_platforms.append(db.groupby('Platform')['Publisher'].unique().to_dict())

[{'PC': array(['Activision', 'Maxis'], dtype=object)},
 {'PC': array(['id Software'], dtype=object)},
 {'PC': array(['Activision', 'Eidos Interactive'], dtype=object),
  'PS': array(['Sony Computer Entertainment', 'Virgin Interactive',
         'Eidos Interactive'], dtype=object)},
 {'DC': array(['Sega', 'Namco Bandai Games', 'Virgin Interactive', 'Ubisoft'],
        dtype=object),
  'PC': array(['Vivendi Games', 'Electronic Arts', 'Havas Interactive',
         'Microsoft Game Studios', 'Activision', 'Interplay',
         'Empire Interactive', 'Eidos Interactive'], dtype=object),
  'PS': array(['Sony Computer Entertainment', 'SquareSoft', 'GT Interactive',
         'Konami Digital Entertainment', 'Virgin Interactive', 'Activision',
         'Atari', 'Take-Two Interactive', 'Electronic Arts',
         'Hasbro Interactive', 'Acclaim Entertainment', 'Codemasters',
         'Crave Entertainment', 'Namco Bandai Games', 'Enix Corporation',
         'Square EA', 'Capcom', 'Eidos Interactive',

In [31]:
publishers = df["Publisher"].unique()

array(['Nintendo', 'Microsoft Game Studios', 'Take-Two Interactive',
       'Sony Computer Entertainment', 'Activision', 'Ubisoft',
       'Bethesda Softworks', 'Electronic Arts', 'SquareSoft',
       'GT Interactive', 'Konami Digital Entertainment', 'Square Enix',
       'Sony Computer Entertainment Europe', 'Virgin Interactive',
       'LucasArts', '505 Games', 'Capcom',
       'Warner Bros. Interactive Entertainment', 'Universal Interactive',
       'RedOctane', 'Atari', 'Eidos Interactive', 'Namco Bandai Games',
       'Vivendi Games', 'MTV Games', 'Sega', 'THQ',
       'Disney Interactive Studios', 'Acclaim Entertainment',
       'Midway Games', 'Deep Silver', 'NCSoft', 'Tecmo Koei',
       'Valve Software', 'Infogrames', 'Mindscape', 'Valve',
       'Hello Games', 'Global Star', 'Gotham Games',
       'Crave Entertainment', 'Hasbro Interactive', 'Codemasters',
       'TDK Mediactive', 'Zoo Games', 'Sony Oznline Entertainment', 'RTL',
       'D3Publisher', 'Unknown', 'Black Label 

In [33]:
df['Name'].loc[df["Publisher"] == 'Nintendo'].nunique() 

290

In [34]:
action_graph = Graph(directed=False) # não-dirigido
action_graph.add_vp('Publisher')
action_graph.add_vp('Games_released')
action_graph.add_ep('strength')

for pub in publishers:
    action_graph.add_vertex(pub)
    action_graph.get_vertex(pub)['Publisher'] = pub
    num_games = df['Name'].loc[df["Publisher"] == pub].nunique() 
    action_graph.get_vertex(pub)['Games_released'] = num_games

# for df in df_list:
#     for pub in publishers:
#         for pub2 in publishers:
#             if pub != pub2 and not action_graph.has_edge(pub, pub2):
#                 action_graph.add_edge(pub, pub2)

In [35]:
for platform_dict in publisher_platforms:
    for platform, publishers in platform_dict.items():
        for i in range(len(publishers)):
            if i != len(publishers)-1:
                next_vertex = i+1
                while(next_vertex < len(publishers)):
                    if(publishers[i] != publishers[next_vertex] and not action_graph.has_edge(publishers[i], publishers[next_vertex])):
                        action_graph.add_edge(publishers[i], publishers[next_vertex])
                        action_graph.get_edge(publishers[i], publishers[next_vertex])['strength'] = 1
                        next_vertex += 1
                    else:
                        action_graph.get_edge(publishers[i], publishers[next_vertex])['strength'] += 1
                        next_vertex += 1

In [36]:
print('número de vértices de g:', action_graph.num_vertices())

número de vértices de g: 264


In [37]:
print('número de arestaas de g:', action_graph.num_edges())

número de arestaas de g: 10970


In [38]:
gt_save(action_graph, 'action_graph.net.gz')

In [39]:
g = gt_load('action_graph.net.gz')
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'action_sfdp.net.gz')

In [40]:
r = netpixi.render('action_sfdp.net.gz', infinite=True);

In [41]:
r.vertex_scale('Games_released', 20, 50)
r.edge_scale('strength', 0.3, 4)
r.vertex_default(color=0xff8674, kscale = 4, bwidth=3, bcolor=0x000000)
r.edge_default(curve1=1, curve2=1)
r.vertex_set_key('Publisher')

# Densidade

In [45]:
num_vertices = action_graph.num_vertices()
num_edges = action_graph.num_edges()

# Como sabemos que nosso grafo não é dirigido, podemos a fórmula a seguir
max_edges = num_vertices * (num_vertices - 1) // 2
density = num_edges / max_edges
print('Densidade do grafo:', density)

Densidade do grafo: 0.3159926258785574


# Centralidade Closeness

In [65]:
from graph_tool import centrality
g = gt_load("action_sfdp.net.gz")

cc = centrality.closeness(g)

In [66]:
g.add_vp('closeness', cc)

In [67]:
gt_save(g, 'graph_closeness_component.net.gz')

In [68]:
g = gt_load('graph_closeness_component.net.gz')

In [69]:
r = netpixi.render('graph_closeness_component.net.gz', infinite=True)

In [70]:
r.vertex_scale('closeness', 10, 50)
r.vertex_set_key('Publisher')

# Centro periferia

In [57]:
import cpnet

from graph_tool import spectral

g = gt_load("action_sfdp.net.gz")

In [62]:
r = netpixi.render('action_sfdp.net.gz', infinite=True);

In [63]:
matrix = spectral.adjacency(g)

# Além de cpnet.Rombach, há outras
# opções de algoritmos contínuos:
# cpnet.MINRES
# cpnet.Rossa

algorithm = cpnet.Rombach()
algorithm.detect(matrix)
c = algorithm.get_coreness()

# Depois do código acima, c é um dicionário
# em que as chaves são índices de vértices e
# valores são os atribuídos pelo algoritmo.

g.add_vp('coreness')
for i, coreness in c.items():
    v = g.get_vertex_by_index(i)
    v['coreness'] = float(coreness)

# Vamos usar coreness como referência para
# o tamanho dos vértices. Estabelecemos 10
# como o menor tamanho possível e 40 como
# o maior tamanho possível. Como o coreness
# está entre 0 e 1, a conta é bem simples.

for v in g.all_vertices():
    r.vertex(v['id'], size=(10 + 40 * v['coreness']))

# Quando a execução desta célula terminar,
# veja como ficou a visualização acima.

In [64]:
r.vertex_set_key('Publisher')